In [ ]:
# Inicialize o Otter
import otter
grader = otter.Notebook("hw12.ipynb")

# Lição de Casa 12: Classificação

**Recurso Útil:**

- [Referência Python](http://data8.org/sp22/python-reference.html): Folha de dicas de métodos úteis para arrays e tabelas usados em Data 8!

**Leitura Recomendada**: 

* [Classificação](https://www.inferentialthinking.com/chapters/17/Classification.html)

Por favor, complete este caderno preenchendo as células fornecidas. Antes de começar, execute a célula a seguir para configurar o caderno importando algumas bibliotecas úteis. Toda vez que você iniciar seu servidor, precisará executar esta célula novamente.

Para todos os problemas em que você deve escrever explicações e frases, você **deve** fornecer sua resposta no espaço designado. **Além disso, ao longo desta lição de casa e de todas as futuras, certifique-se de não reatribuir variáveis ao longo do caderno!** Por exemplo, se você usar `max_temperature` na sua resposta para uma pergunta, não o reatribua mais tarde. Caso contrário, você falhará em testes que pensou ter passado anteriormente!

**Nota: Esta lição de casa tem testes ocultos. Isso significa que, mesmo que os testes digam que 100% passaram, não significa que sua nota final será 100%. Nós executaremos mais testes para correção uma vez que todos entreguem a lição de casa.**

Compartilhar diretamente as respostas não é permitido, mas discutir problemas com a equipe do curso ou com outros alunos é encorajado.

Você deve começar cedo para que tenha tempo de obter ajuda se estiver com dificuldades.

In [ ]:
# Não altere esta célula; apenas execute-a.

import numpy as np
from datascience import *
import d8error

# Estas linhas fazem alguma mágica de plotagem sofisticada
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore')  # Ignora avisos simples
from datetime import datetime

## 1. Coordenadas das Escolas da Área da Baía com Classificação

Bem-vindo à Lição de Casa 12! Esta lição de casa é sobre a classificação k-Vizinhos Mais Próximos (k-NN). Como este tópico é abordado em profundidade no Projeto 3, o propósito desta lição de casa é reforçar os conceitos básicos deste método. Você pode e deve reutilizar muito do código que escreveu para o Projeto 3 nesta lição de casa, ou usar código desta lição de casa no Projeto 3!

### Nossos Queridos Vizinhos

Carol está tentando classificar estudantes como frequentadores da UC Berkeley ou como frequentadores da Universidade de Stanford. Para classificar os estudantes, Carol tem acesso às coordenadas do local onde eles moram durante o ano letivo. Primeiro, carregue a tabela `coordinates`.

In [ ]:
# Apenas execute esta célula
coordinates = Table.read_table('coordinates.csv')
coordinates.show(5)

Como de costume, vamos investigar nossos dados visualmente antes de realizar qualquer tipo de análise numérica.

In [ ]:
# Apenas execute esta célula
coordinates.scatter("longitude", "latitude", group="school")

Os locais dos pontos neste gráfico de dispersão podem ser familiares - execute a célula a seguir para ver a que eles correspondem.

In [ ]:
# Apenas execute esta célula
colors = {"Berkeley":"blue", "Stanford":"red"}
t = Table().with_columns("lat", coordinates.column(0), 
                                      "lon", coordinates.column(1), 
                                      "color", coordinates.apply(colors.get, 2)
                        )
Circle.map_table(t, radius=5, fill_opacity=1)

**Questão 1.1.** Vamos começar a implementar o algoritmo k-Vizinhos Mais Próximos. Defina a função `distance`, que recebe dois argumentos: um array de características numéricas (`arr1`) e um array diferente de características numéricas (`arr2`). A função deve retornar a [distância Euclidiana](https://pt.wikipedia.org/wiki/Dist%C3%A2ncia_euclidiana) entre os dois arrays. A distância Euclidiana é frequentemente referida como a fórmula da distância em linha reta que você pode ter aprendido anteriormente. **(10 pontos)**

In [ ]:
def distance(arr1, arr2):
    ...

# Não altere o código da célula abaixo desta linha
distance_example = distance(make_array(1, 2, 3), make_array(4, 5, 6))
distance_example

In [ ]:
grader.check("q1_1")

### Dividindo o Conjunto de Dados
Faremos dois tipos diferentes de coisas com o conjunto de dados `coordinates`:

1. Construiremos um classificador usando coordenadas para as quais conhecemos o rótulo associado; isso o ensinará a reconhecer rótulos de valores de coordenadas similares. Esse processo é conhecido como *treinamento*.
2. Avaliaremos ou *testaremos* a precisão do classificador que construímos em dados que não vimos

**Questão 1.2.** A seguir, vamos dividir nosso conjunto de dados em um conjunto de treinamento e um conjunto de teste. Como `coordinates` tem 100 linhas, vamos criar um conjunto de treinamento com as primeiras 75 linhas e um conjunto de teste com as 25 linhas restantes. Lembre-se de que a atribuição a cada grupo deve ser aleatória, então devemos embaralhar a tabela primeiro. **(10 pontos)**

*Dica:* Como primeiro passo, podemos embaralhar todas as linhas e, em seguida, usar a função `tbl.take` para dividir as linhas de cada tabela.

In [ ]:
shuffled_table = ...
train = ...
test = ...

print("Training set:\t",   train.num_rows, "examples")
print("Test set:\t",       test.num_rows, "examples")
train.show(5), test.show(5);

In [ ]:
grader.check("q1_2")

**Questão 1.3.** Atribua `features` a um array de nomes de colunas (strings) das características da tabela `coordinates`. **(10 pontos)**

*Dica:* Quais dos nomes de colunas na tabela `coordinates` são as características, e quais dos nomes de colunas correspondem à classe que estamos tentando prever?

*Dica:* Não é necessário modificar nenhuma tabela, apenas crie manualmente um array com os nomes das características!

In [ ]:
features = ...
features

In [ ]:
grader.check("q1_3")

**Questão 1.4.** Agora defina a função `classify`. Esta função deve receber uma `test_row` de uma tabela como `test` e classificá-la usando os k-Vizinhos Mais Próximos com base nas `features` corretas e nos dados em `train`. Uma revisão sobre k-Vizinhos Mais Próximos pode ser encontrada [aqui](https://www.inferentialthinking.com/chapters/17/4/Implementing_the_Classifier.html). **(10 pontos)**


*Dica 1:* A função `distance` que definimos anteriormente recebe arrays como entrada, então use a função `row_to_array` que definimos para você para converter linhas em arrays de características.

*Dica 2:* O código esqueleto que fornecemos itera por cada linha no conjunto de treinamento.

In [ ]:
def row_to_array(row, features):
    """Converts a row to an array of its features."""
    arr = make_array()
    for feature in features:
        arr = np.append(arr, row.item(feature))
    return arr

def classify(test_row, k, train):
    test_row_features_array = row_to_array(test_row, features)
    distances = make_array()
    for train_row in train.rows:
        train_row_features_array = ...
        row_distance = ...
        distances = ...
    train_with_distances = ...
    nearest_neighbors = ...
    most_common_label = ...
    ...

# Não altere o código abaixo
first_test = classify(test.row(0), 5, train)
first_test

In [ ]:
grader.check("q1_4")

**Questão 1.5.** Defina a função `three_classify` que recebe uma `row` de `test` como argumento e classifica a linha usando 3-Vizinhos Mais Próximos. Use esta função para encontrar a `accuracy` (precisão) de um classificador 3-NN no conjunto `test`. `accuracy` deve ser uma proporção (não uma porcentagem) das escolas que foram corretamente previstas. **(10 pontos)**


*Dica:* Você deve usar uma função que acabou de criar!

*Nota:* Geralmente, antes de usar um classificador em um conjunto de teste, classificaríamos primeiro em um conjunto de "validação", o qual poderíamos então modificar nosso conjunto de treinamento novamente, se necessário, antes de realmente testar no conjunto de teste. Você não precisa fazer isso para esta questão, mas aprenderá mais sobre isso em Data 100.

In [ ]:
def three_classify(row):
    ...

test_with_prediction = ...
labels_correct = ...
accuracy = ...
accuracy

In [ ]:
grader.check("q1_5")

**Questão 1.6.** Existem 77 linhas de estudantes da UC Berkeley e 23 linhas de estudantes da Universidade de Stanford na tabela `coordinates`. Se usássemos a tabela `coordinates` inteira como o conjunto de treinamento, qual é o menor valor de k que garantiria que um classificador k-Vizinhos Mais Próximos *sempre* preveria Berkeley como a classe? Atribua o valor a `k`. **(10 pontos)**

In [ ]:
k = ...
k

In [ ]:
grader.check("q1_6")

<!-- BEGIN QUESTION -->

**Questão 1.7.** Por que dividimos nossos dados em um conjunto de treinamento e um conjunto de teste? Qual é o propósito de um conjunto de teste, e por que queremos usar o conjunto de teste apenas uma vez? Explique sua resposta em 3 frases ou menos. **(10 pontos)**

*Dica:* Confira esta [seção](https://inferentialthinking.com/chapters/17/2/Training_and_Testing.html) no livro didático.

_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Questão 1.8.** Por que usamos um número ímpar para `k` em k-NN? Explique. **(10 pontos)**

_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

**Questão 1.9.0. Configuração**

Thomas elaborou um esquema para dividir o conjunto de teste e o conjunto de treinamento. Para cada linha de `coordinates`:
- Linhas de estudantes de Stanford têm 50% de chance de serem colocadas no conjunto de treinamento e 50% de chance de serem colocadas no conjunto de teste.
- Linhas de estudantes de Berkeley têm 80% de chance de serem colocadas no conjunto de treinamento e 20% de chance de serem colocadas no conjunto de teste.

*Dica 1:* Lembre-se de que há 77 estudantes de Berkeley e 23 estudantes de Stanford em coordinates.

*Dica 2:* O sobrenome de Thomas é Bayes. (Então, a [Seção 18.1](https://inferentialthinking.com/chapters/18/1/More_Likely_than_Not_Binary_Classifier.html#bayes-rule) do livro didático pode ser útil aqui!)

*Dica 3:* O seguinte diagrama de árvore pode ser útil nas Questões 1.9.1 e 1.9.2!

<img src="tree_diagram.png" width="450">

**Questão 1.9.1.** Dado que uma linha está no conjunto de teste, qual é a probabilidade de que ela corresponda a um estudante de Stanford? Atribua essa probabilidade a `prob_furd`. **(10 pontos)**

In [ ]:
prob_furd = ...
prob_furd

In [ ]:
grader.check("q1_9_1")

**Questão 1.9.2.** Dado que uma linha é de Stanford, qual é a probabilidade de que o estudante esteja no conjunto de teste? Atribua essa probabilidade a `prob_test`. **(10 pontos)**

In [ ]:
prob_test = ...
prob_test

In [ ]:
grader.check("q1_9_2")

## (OPCIONAL, FORA DO ESCOPO): k-NN para Classificação Não Binária

**ISSO NÃO ESTÁ NO ESCOPO**. Não há testes de correção automática para isso ou código para você escrever. Isso apenas depende da função `classify` na Questão 1.4. Siga em frente e leia esta seção e execute as células a seguir!

Nesta classe, ensinamos você a usar o algoritmo k-NN para classificar dados como uma de duas classes. No entanto, grande parte dos dados que você encontrará no mundo real não se encaixará perfeitamente em uma de duas categorias.

**Como podemos classificar dados com classes não binárias?** Acontece que ainda podemos usar k-NN! Ou seja, encontramos a distância entre um ponto e todos os seus vizinhos, encontramos os vizinhos mais próximos e fazemos uma votação majoritária entre os vizinhos para determinar a classe deste ponto.

A única diferença é que agora os pontos vizinhos têm mais de duas classes possíveis. Isso introduz dificuldade porque agora não temos como garantir que não encontraremos empates entre as classes. No caso de encontrarmos um empate, podemos simplesmente escolher arbitrariamente uma das classes.

Na verdade, você nem precisa modificar o código que escreveu antes para habilitar a classificação multiclasse!

Vamos adicionar mais dados à nossa tabela de treinamento, desta vez para outra classe de estudantes, estudantes do San Jose Community College (SJCC).

In [ ]:
coordinates_multi = coordinates.with_rows([
                              [37.304346, -121.915401, "SJCC"],
                              [37.316275, -121.913879, "SJCC"],
                              [37.409435, -121.951379, "SJCC"],
                              [37.349387, -121.960771, "SJCC"],
                              [37.329083, -121.928479, "SJCC"],
                              [37.313017, -121.866730, "SJCC"],
                              [37.346525, -121.894767, "SJCC"],
                              [37.364157, -121.955717, "SJCC"],
                              [37.383362, -121.925776, "SJCC"],
                              [37.329545, -121.880639, "SJCC"]                             
])

In [ ]:
classify(coordinates_multi.row(0), 5, coordinates_multi)

In [ ]:
classify(coordinates_multi.row(91), 5, coordinates_multi)

In [ ]:
classify(coordinates_multi.row(105), 5, coordinates_multi)

Nosso classificador pode classificar linhas como pertencentes a uma de três classes!

A classificação é um dos campos mais importantes em estatística, ciência de dados e aprendizado de máquina. Existem milhares de algoritmos de classificação diferentes e modificações de algoritmos! Há muitos que você aprenderá se continuar no caminho para se tornar um cientista de dados!

Você concluiu a Lição de Casa 12! :,)

**Passos importantes para a submissão:** 
1. Execute os testes e verifique se todos passaram.
2. Escolha **Salvar Caderno** no menu **Arquivo**, depois **execute a célula final**.
3. Clique no link para baixar o arquivo zip.
4. Então, submeta o arquivo zip para a tarefa correspondente de acordo com as instruções do seu instrutor.

**É sua responsabilidade garantir que seu trabalho esteja salvo antes de executar a última célula.**

## Submissão

Certifique-se de ter executado todas as células no seu caderno em ordem antes de executar a célula abaixo, para que todas as imagens/gráficos apareçam na saída. A célula abaixo irá gerar um arquivo zip para você submeter. **Por favor, salve antes de exportar!**

In [ ]:
# Salve seu caderno primeiro, depois execute esta célula para exportar sua submissão.
grader.export(pdf=False, run_tests=True)